# Get Initialized Data

In [1]:
from nlptext.base import BasicObject

########### Wiki ###########
CORPUSPath = 'corpus/wiki_cn_sample/'

Corpus2GroupMethod = '.txt'

Group2TextMethod   = 'line'

Text2SentMethod  = 'whole'

Sent2TokenMethod = 'pos'
TOKENLevel = 'word'
min_token_freq = 1

use_hyper = ['pos']


anno = False
anno_keywords = {}


BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = False, anno_keywords = anno_keywords)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/wiki_cn_sample/sample_wiki_smp.txt


Loading model cost 0.562 seconds.
Prefix dict has been built succesfully.


Total Num of All    Tokens 8407
Total Num of Unique Tokens 2000
CORPUS	it is Dumped into file: data/wiki_cn_sample/word/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/wiki_cn_sample/word/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/wiki_cn_sample/word/Pyramid/TEXT.p
TEXT	the length of it is   : 100
SENT	it is Dumped into file: data/wiki_cn_sample/word/Pyramid/SENT.p
SENT	the length of it is   : 100
TOKEN	it is Dumped into file: data/wiki_cn_sample/word/Pyramid/TOKEN.p
TOKEN	the length of it is   : 8407
**************************************** 

pos-bioes	is Dumped into file: data/wiki_cn_sample/word/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/wiki_cn_sample/word/Vocab/pos-bioes.tsv
token   	is Dumped into file: data/wiki_cn_sample/word/Vocab/token.voc
token   	the length of it is   : 2000
		Write to: data/wiki_cn_sample/word/Vocab/token.tsv
****************************************


# Get Chunk Information

In [2]:
import numpy as np
from nlptext.sentence import Sentence
from nlptext.text import Text

def get_chunk_info(BasicObject, Object, BATCH_MAX_NUM = 10000):
    
    if 'sent' in Object.lower():
        Object = Sentence
        Objects_Num = BasicObject.SENT['length']
        
    elif 'text' in Object.lower():
        Object = Text
        Objects_Num = BasicObject.TEXT['length']
    else:
        raise('No Such Object:', Object)
        
        
    # channel = 'token' 
    # chunkidx_2_endbyteidx = [] 
    
    chunkidx_2_endbyteidxs = {}
    for channel in BasicObject.Channel_Hyper_Path:
        chunkidx_2_endbyteidxs[channel] = []
        
    chunkidx_2_cumlengoftexts = []
    # this can be removed
    chunkidx_2_length_count = []


    current_chunk_lengoftexts = []
    current_chunk_length_count = 0
    
    for objectidx in range(Objects_Num):
        # the objectidx is the locidx
        text = Object(objectidx) 
        text_token_num = text.length
        length_with_new_text = current_chunk_length_count + text_token_num
        if  length_with_new_text > BATCH_MAX_NUM:
            # pay attention to this operation
            # this text is exclusive to current chunk, so the end of chunk is the start of this text
            
            # endbyteidx, _ = text.start_end_position(channel) 
            # chunkidx_2_endbyteidx.append(endbyteidx)
            
            for channel in chunkidx_2_endbyteidxs:
                endbyteidx, _ = text.start_end_position(channel) 
                chunkidx_2_endbyteidxs[channel].append(endbyteidx)
                
            # also current_chunk_lengoftexts doesn't include the information of this text
            chunkidx_2_cumlengoftexts.append(np.cumsum(current_chunk_lengoftexts))

            # records the chunk length, this can be removed
            chunkidx_2_length_count.append(current_chunk_length_count)

            # in the following block, the new text matters.
            # get the new the current chunk information
            current_chunk_length_count = text_token_num
            current_chunk_lengoftexts = [text_token_num]

        else:
            # update current_chunk_length_count when it is <= 10000
            current_chunk_length_count = length_with_new_text

            # append the object's length in the current_chunk_lengoftexts
            current_chunk_lengoftexts.append(text_token_num)


    # when the loop is over, we still need the to append 
    # the last small chunk into the total chunks.
    # here, the object is the smallest object.

    # notice, that this text is included in the last chuck
    # _, endbyteidx = text.start_end_position(channel) 
    # chunkidx_2_endbyteidx.append(endbyteidx)
    
    for channel in chunkidx_2_endbyteidxs:
        _, endbyteidx = text.start_end_position(channel) 
        chunkidx_2_endbyteidxs[channel].append(endbyteidx)
                

    # save the txtidx2endtokenidx, derived from lengoftexts
    chunkidx_2_cumlengoftexts.append(np.cumsum(current_chunk_lengoftexts))

    # records the chunk length, this can be removed
    chunkidx_2_length_count.append(current_chunk_length_count)

    return chunkidx_2_endbyteidxs, chunkidx_2_cumlengoftexts, chunkidx_2_length_count

In [3]:
from nlptext.text import Text

Object = 'text'
get_chunk_info(BasicObject, Object, BATCH_MAX_NUM = 10000)

({'token': [57833, 115292, 171624, 231761, 243352],
  'pos': [35378, 70593, 105752, 141103, 147910]},
 [array([  79,  184,  291,  425,  562,  626,  686,  720,  782,  869,  980,
         1177, 1227, 1320, 1438, 1558, 1708, 1834, 1969, 2075, 2243, 2454,
         2571, 2587, 2650, 2690, 2760, 2871, 2990, 3013, 3026, 3074, 3293,
         3451, 3581, 3622, 3748, 3923, 4008, 4080, 4196, 4205, 4238, 4290,
         4327, 4363, 4408, 4428, 4442, 4478, 4544, 4635, 4705, 4797, 4875,
         5059, 5088, 5194, 5325, 5451, 5561, 5672, 5702, 5758, 5835, 5918,
         5950, 6104, 6146, 6260, 6391, 6478, 6613, 6656, 6782, 6832, 6837,
         6945, 6988, 7055, 7206, 7304, 7349, 7422, 7580, 7623, 7711, 7794,
         7804, 7835, 7912, 7930, 7965, 8018, 8092, 8144, 8198, 8245, 8372,
         8414, 8459, 8484, 8555, 8564, 8572, 8575, 8578, 8581, 8586, 8588,
         8591, 8593, 8596, 8599, 8602, 8604, 8606, 8610, 8613, 8616, 8618,
         8620, 8622, 8704, 8985, 9095, 9152, 9226, 9378, 9521, 9632, 9687

In [4]:
chunkidx_2_endbyteidxs, chunkidx_2_cumlengoftexts, chunkidx_2_length_count = get_chunk_info(BasicObject, Object, BATCH_MAX_NUM = 10000)

chunkidx_2_endbyteidxs

{'token': [57833, 115292, 171624, 231761, 243352],
 'pos': [35378, 70593, 105752, 141103, 147910]}

In [5]:
chunkidx_2_cumlengoftexts

[array([  79,  184,  291,  425,  562,  626,  686,  720,  782,  869,  980,
        1177, 1227, 1320, 1438, 1558, 1708, 1834, 1969, 2075, 2243, 2454,
        2571, 2587, 2650, 2690, 2760, 2871, 2990, 3013, 3026, 3074, 3293,
        3451, 3581, 3622, 3748, 3923, 4008, 4080, 4196, 4205, 4238, 4290,
        4327, 4363, 4408, 4428, 4442, 4478, 4544, 4635, 4705, 4797, 4875,
        5059, 5088, 5194, 5325, 5451, 5561, 5672, 5702, 5758, 5835, 5918,
        5950, 6104, 6146, 6260, 6391, 6478, 6613, 6656, 6782, 6832, 6837,
        6945, 6988, 7055, 7206, 7304, 7349, 7422, 7580, 7623, 7711, 7794,
        7804, 7835, 7912, 7930, 7965, 8018, 8092, 8144, 8198, 8245, 8372,
        8414, 8459, 8484, 8555, 8564, 8572, 8575, 8578, 8581, 8586, 8588,
        8591, 8593, 8596, 8599, 8602, 8604, 8606, 8610, 8613, 8616, 8618,
        8620, 8622, 8704, 8985, 9095, 9152, 9226, 9378, 9521, 9632, 9687,
        9793, 9837, 9877, 9895, 9965, 9986]),
 array([ 108,  173,  505,  608,  664,  804,  897,  944, 1029, 1051

In [6]:
new_count = [i[-1] for i in chunkidx_2_cumlengoftexts]
list(zip(new_count, chunkidx_2_length_count))

[(9986, 9986), (9982, 9982), (9963, 9963), (9985, 9985), (1927, 1927)]

In [7]:
path = BasicObject.Channel_Hyper_Path['token']
print(path)

data/wiki/word/Pyramid/_file/token.txt


In [8]:
# for example, let's check a chunk with its information
from nlptext.utils.pyramid import read_file_chunk_string
import re

chunk_idx = 3
channel = 'token'


start_position = 0 if chunk_idx == 0 else chunkidx_2_endbyteidxs[channel][chunk_idx - 1] 
end_postion = chunkidx_2_endbyteidxs[channel][chunk_idx] 

chunk_string = read_file_chunk_string(path, start_position, end_postion)
print(chunk_string[:200] + '....')



实用 经济学 理 对于 公共政策 的 研究 大多 聚焦 于 如何 增进 一个 经济 的 效率 , 找出 如何 调整 社会 组织 以 达成 最 有效 的 资源 生产 , 被 认为 是 「 经济学 的 本质 」
专精 是 一种 无论 在 经济 理论 上 和 观察 上 都 已经 被 认定 是 经济 效率 重要 来源 的 现象 , 不同 的 个体 和 国家 可能 对 生产 各种 产品 和 服务 有 不同 ....


In [9]:
print(len(re.split(' |\n', chunk_string)))
print(chunkidx_2_length_count[chunk_idx])

9985
9985
